In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

In [ ]:
delimiter = 9999.0

maxDistance = 20
minDistance = 0
sensor2CenterDistance = 10.3

dtheta = 1.8
dz = 0.1

raw_data = []

with open('farmer.txt') as f:
    [raw_data.append(float(line.strip())) for line in f.readlines()]

raw_data = np.array(raw_data)
print(raw_data)

raw_data = np.array(raw_data)
raw_data[raw_data<0] = 0
indices = np.where(raw_data==delimiter)
indices = indices[0]

r = np.zeros((len(indices)-1, indices[0]))

r[0,:] = raw_data[0:indices[0]]
for i in range(1,len(indices)-1):
    r[i,:] = raw_data[indices[i-1]+1:indices[i]]

# distances converted to radius measurements
r = sensor2CenterDistance - r

# matrix of theta values same shape as r
theta = np.arange(0, r.shape[1]*dtheta, dtheta)
theta = np.tile(theta, (r.shape[0], 1))

# matrix of z values same shape as r
z = np.arange(start=0, stop=r.shape[0]*dz, step=dz)
z = np.transpose(np.tile(z, (r.shape[1], 1)))

# cylindrical to cartesian 
x = r*np.cos(np.deg2rad(theta))
y = r*np.sin(np.deg2rad(theta))


In [ ]:
for i in range(0, x.shape[0]):
    if np.sum(np.isnan(x[i, :]))==x.shape[1]:
        x[i:, :]=[]
        y[i:, :]= []
        z[i:, :]= []
        break

for i in range(0, x.shape[0]):
    lastIdx = np.argmax(~np.isnan(x[i,:]))
    lastX = x[i, lastIdx]
    lastY = y[i, lastIdx]
    for j in range(0, x.shape[1]):
        if ~np.isnan(x[i,j]):
            lastX = x[i,j]
            lastY = y[i,j]
        else:
            x[i,j] = lastX
            y[i,j] = lastY

    
interpIdx = np.arange(0, x.shape[0], 1)
xInterp = x[interpIdx, :]
yInterp = y[interpIdx, :]
zInterp = z[interpIdx, :]

window_size = 2

h = np.ones((window_size, window_size))/ (window_size**2)

# symmetric padding along rows
xInterp = np.pad(xInterp, ((0,0), (window_size, window_size)), mode='symmetric')
yInterp = np.pad(yInterp, ((0,0), (window_size, window_size)), mode='symmetric')

xInterp = convolve2d(xInterp, h)
yInterp = convolve2d(yInterp, h)

xInterp = xInterp[:, window_size:-window_size]
yInterp = yInterp[:, window_size:-window_size]


xInterp[:, -1]=xInterp[:, 0]
yInterp[:, -1]=yInterp[:, 0]
zInterp[:, -1]=zInterp[:, 0]

xTop = np.mean(xInterp[-1, :])
yTop = np.mean(yInterp[-1, :])
zTop = zInterp[-1, 0]

pc_list = list(zip(xInterp.flatten(), yInterp.flatten(), zInterp.flatten()))
pc_list.append((xTop, yTop, zTop))

pc_data = np.array(pc_list)

In [ ]:
print(z.shape)
print(x.shape)
print(y.shape)

# Plotting - with PyVista

In [ ]:
import numpy as np
import pyvista as pv
from scipy.signal import convolve2d
from matplotlib import pyplot as plt

In [ ]:
# scan conversion variables
maxDistance = 18.0 # [cm]
minDistance =  2.0 # [cm]
sensor2CenterDistance = 8.7 #[cm]

txPerScan = 5
delimiter = 1234.00
dtheta = 18
dz = 0.5

In [ ]:
raw_data = np.loadtxt('pleasework.txt') 
raw_data[raw_data<minDistance] = None
indices = np.where(raw_data==delimiter)
raw_data[raw_data>maxDistance]= None #maxDistance
indices = indices[0]

In [ ]:
neg_pos = theta[np.where(r<0)] 
indices = np.where(r<0)
indices



In [ ]:
r_unprocessed = np.zeros((len(indices)-1, indices[0]//txPerScan, txPerScan))
r = np.zeros((len(indices)-1, indices[0]//txPerScan))

for z in range(r_unprocessed.shape[0]):
    for theta in range(r_unprocessed.shape[1]):
        for scan in range(r_unprocessed.shape[2]):
            r_unprocessed[z, theta, scan] = raw_data[z*r_unprocessed.shape[1] + theta*r_unprocessed.shape[2] + scan]

# averaging
for z in range(0, r_unprocessed.shape[0]):
    for theta in range(0, r_unprocessed.shape[1]):
        r[z, theta] = np.mean(r_unprocessed[z, theta, (r_unprocessed[z,theta,:]!=None)])


In [ ]:
r_unprocessed

In [ ]:
r

In [ ]:
# distances converted to radius measurements
r = sensor2CenterDistance - r

# matrix of theta values same shape as r
theta = np.arange(0, r.shape[1]*dtheta, dtheta)
theta = np.tile(theta, (r.shape[0], 1))

# matrix of z values same shape as r
z = np.arange(start=0, stop=r.shape[0]*dz, step=dz)
z = np.transpose(np.tile(z, (r.shape[1], 1)))

# cylindrical to cartesian 
x = r*np.cos(np.deg2rad(theta))
y = r*np.sin(np.deg2rad(theta))


pc_data_unfiltered=np.array( list(zip(x.flatten(), y.flatten(), z.flatten())))


In [ ]:
len(r)

In [ ]:
len(theta)

In [ ]:
layer_theta = np.split(theta,51)
layer_r = np.split(r, 51)

In [ ]:
plt.scatter(x,z,s=100)

In [ ]:
unfiltered_cloud = pv.PolyData(pc_data_unfiltered)
unfiltered_cloud.plot()

In [ ]:
# filtering
for i in range(0, x.shape[0]):
    if np.sum(np.isnan(x[i, :]))==x.shape[1]:
        x[i:, :]=[]
        y[i:, :]= []
        z[i:, :]= []
        break

for i in range(0, x.shape[0]):
    lastIdx = np.argmax(~np.isnan(x[i,:]))
    lastX = x[i, lastIdx]
    lastY = y[i, lastIdx]
    for j in range(0, x.shape[1]):
        if ~np.isnan(x[i,j]):
            lastX = x[i,j]
            lastY = y[i,j]
        else:
            x[i,j] = lastX
            y[i,j] = lastY


interpIdx = np.arange(0, x.shape[0], 1)
xInterp = x[interpIdx, :]
yInterp = y[interpIdx, :]
zInterp = z[interpIdx, :]

window_size = 2

h = np.ones((window_size, window_size))/ (window_size**2)

# symmetric padding along rows
xInterp = np.pad(xInterp, ((0,0), (window_size, window_size)), mode='symmetric')
yInterp = np.pad(yInterp, ((0,0), (window_size, window_size)), mode='symmetric')

xInterp = convolve2d(xInterp, h)
yInterp = convolve2d(yInterp, h)

xInterp = xInterp[:, window_size:-window_size]
yInterp = yInterp[:, window_size:-window_size]


xInterp[:, -1]=xInterp[:, 0]
yInterp[:, -1]=yInterp[:, 0]
zInterp[:, -1]=zInterp[:, 0]

xTop = np.mean(xInterp[-1, :])
yTop = np.mean(yInterp[-1, :])
zTop = zInterp[-1, 0]

pc_list = list(zip(xInterp.flatten(), yInterp.flatten(), zInterp.flatten()))
pc_list.append((xTop, yTop, zTop))

pc_data = np.array(pc_list)

# cloud = pv.PolyData(pc_data)
cloud = pv.PolyData(pc_data)
cloud.plot()

# if input("create a mesh object? (yes/no): ")=='yes':
#     volume = cloud.delaunay_3d()
#     shell = volume.extract_geometry()
#     shell.plot()

# if input("Plot the mesh of the unfiltered? (yes/no): ")=='yes':
#     uf_cloud = pv.PolyData(pc_data_unfiltered)
#     volume =uf_cloud.delaunay_3d()
#     shell = volume.extract_geometry()
#     shell.plot()

# Center Test

In [ ]:
raw_data = np.loadtxt('center_test.txt') 

raw_data = np.array(raw_data)
raw_data[raw_data<minDistance] = None
indices = np.where(raw_data==delimiter)
raw_data[raw_data>maxDistance]= None #maxDistance
indices = indices[0]



In [ ]:
raw_data.shape

In [ ]:
r_unprocessed = np.zeros((len(indices)-1, int(indices[0]/txPerScan), txPerScan))
r = np.zeros((len(indices)-1, indices[0]))

for z in range(r_unprocessed.shape[0]):
    for theta in range(r_unprocessed.shape[1]):
        for scan in range(r_unprocessed.shape[2]):
            try:
                r_unprocessed[z, theta, scan] = raw_data[z*r_unprocessed.shape[1] + theta*r_unprocessed.shape[2] + scan]
            except IndexError:
                print(f'z = {z}, theta = {theta}, scan = {scan}')


In [18]:
import numpy as np
import matplotlib.pyplot as plt

raw_data = np.loadtxt('2D_Scan_Data.txt')

minDistance = 0
maxDistance = 30

raw_data = np.array(raw_data)
raw_data[raw_data<minDistance] = None
raw_data[raw_data>maxDistance] = None

raw_data

array([ nan,  nan,  nan, 3.88, 4.06,  nan,  nan,  nan, 4.28, 4.47,  nan,
        nan,  nan, 4.79, 4.47,  nan,  nan,  nan, 5.68, 5.3 ,  nan,  nan,
        nan, 6.02, 5.2 ,  nan,  nan,  nan, 7.74, 6.21,  nan,  nan,  nan,
        nan, 7.09,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
       8.57,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan])

In [19]:
depths = np.reshape(raw_data, (len(raw_data)//5, 5))
depths = depths[::-1, :]

array([[ nan,  nan,  nan, 3.88, 4.06],
       [ nan,  nan,  nan, 4.28, 4.47],
       [ nan,  nan,  nan, 4.79, 4.47],
       [ nan,  nan,  nan, 5.68, 5.3 ],
       [ nan,  nan,  nan, 6.02, 5.2 ],
       [ nan,  nan,  nan, 7.74, 6.21],
       [ nan,  nan,  nan,  nan, 7.09],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan, 8.57],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan]])

In [15]:
depths = np.transpose(depths)
depths.shape

(5, 20)

In [17]:
depths

array([[  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan],
       [ 6.89,  6.89,  7.4 ,  6.89,  6.89,  6.99,  7.  ,  6.89,  6.99,
         7.29,  6.94,  6.83,  6.95,  7.34,  7.24,  7.7 ,  7.8 ,  8.57,
        10.73,   nan],
       [ 6.48,  6.95,  6.99,  6.89,  6.9 ,  6.99,  6.99,  6.89,  6.94,
         6.85,  6.95,  6.85,  6.95,  7.34,  7.34,  7.65,  8.21,  8.57,
         8.62,  9.13],
       [ 7.04,  7.02,  7.53,  7.04,  7.02,  7.09,  7.09,  6.99,  7.09,
         6.99,  7.04,  6.53,  7.04,  7.04,  7.45,  7.33,  7.87,  7.77,
         8.67,  8.77]])